# Getting Started with Chassis
Run this notebook to execute your first Chassis job with a pre-trained Scikit-learn Logistic Regression Classifier.

**Note:** To follow along with the example code below, you must create a free [Docker Hub](https://hub.docker.com/signup) account if you do not already have one. We also recommend setting up a virtual environment to run this notebook. Chassis currently supports Python 3.6 and above

In [ ]:
!pip install chassisml scikit-learn numpy

In [ ]:
import json
import pickle
import chassisml
import numpy as np

In [ ]:
# load model
model = pickle.load(open("./model.pkl", "rb"))

# define process function
def process(input_bytes):
    inputs = np.array(json.loads(input_bytes))
    inference_results = model.predict(inputs)
    structured_results = []
    for inference_result in inference_results:
        structured_output = {
            "data": {
                "result": {"classPredictions": [{"class": str(inference_result), "score": str(1)}]}
            }
        }
        structured_results.append(structured_output)
    return structured_results

In [ ]:
# connect to Chassis client
chassis_client = chassisml.ChassisClient("https://chassis.app.modzy.com/")

# create Chassis model
chassis_model = chassis_client.create_model(process_fn=process)

# test Chassis model
sample_filepath = './digits_sample.json'
results = chassis_model.test(sample_filepath)
print(results)

In [ ]:
# publish model to Dockerhub
docker_user = "<insert-Docker Hub username>"
docker_pass = "<insert-Docker Hub password>"
model_name = "My First Chassis Model"

response = chassis_model.publish(
    model_name=model_name,
    model_version="0.0.1",
    registry_user=docker_user,
    registry_pass=docker_pass
)

In [ ]:
# wait for job to complete and print result
job_id = response.get('job_id')
final_status = chassis_client.block_until_complete(job_id)

In [ ]:
if final_status['status']['succeeded'] == 1:
    print("Job Completed. View your new container image here: https://hub.docker.com/repository/docker/{}/{}".format(docker_user, "-".join(model_name.lower().split(" "))))
else:
    print("Job Failed. See logs below:\n\n{}".format(final_status['logs']))